<a href="https://colab.research.google.com/github/bbcx-investments/notebooks/blob/main/fixed_income/rate_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from scipy.optimize import minimize
import numpy as np
import pandas as pd

def price(prds, **kwargs):
    c = 100 * kwargs['coupon'] / 2
    maturity = kwargs['maturity']
    if 'yld' in kwargs.keys():
        yld = kwargs['yld']
        n = int(2 * maturity)
        return c * np.sum((1 + yld/2) ** np.arange(-1, -n - 1, -1)) + 100 / (1 + yld/2) ** n
    else:
        n = int(prds * maturity)
        pphy = int(prds/2)  # periods per half year
        spots = np.array(kwargs['spots'])
        # pv_factors = (1 + spots / 2) ** (-np.arange(1, len(spots) + 1))
        pv_factors = (1 + spots / prds) ** (-np.arange(1, len(spots) + 1))
        coupons = np.zeros(n)
        coupons[(pphy - 1)::pphy] = c
        return np.sum(coupons*pv_factors[:len(coupons)]) + 100*pv_factors[n-1]

def forward_rates(prds, spots):
    pphy = int(prds / 2)  # periods per half year
    future_factors = (1 + spots / 2) ** (np.arange(1, len(spots) + 1) / pphy)
    change_logs = np.diff(np.log(future_factors))
    f = (np.exp(change_logs)-1) * prds
    return np.concatenate(([spots[0]], f))

def objective(prds, bonds, spots):
    prices = [price(prds, **bond) for bond in bonds]
    phats = [price(prds=prds, maturity=bond['maturity'], coupon=bond['coupon'], spots=spots) for bond in bonds]
    errors = [np.log(phat/p) for phat, p in zip(prices, phats)]
    sse = np.sum([e**2 for e in errors])
    forwards = forward_rates(prds, spots)
    diffs = np.sum(np.diff(np.log(1+forwards/prds))**2)
    return sse + 0.5*diffs

def spot_rates(prds, bonds):
    maturities = [bond['maturity'] for bond in bonds]
    n = int(np.max(maturities) * prds)
    result = minimize(lambda x: objective(prds, bonds, x), [0.05]*n)
    return result.x if result.success==True else np.nan

def rateTree(r, sigma, dt, phis):
    delta = sigma*np.sqrt(dt)
    return [[r + phi + delta * (i - 2 * j) for j in range(i + 1)] for i, phi in enumerate(phis)]

def phi(sigma, prds, n, forwards):
    dt = 1/prds
    m = len(forwards)
    f = forwards[:n+1] if m>=n+1 else np.concatenate((forwards, [forwards[-1]]*(n+1-m)))
    term1 = np.log(1+f*dt)
    a = np.exp(sigma*dt**(3/2)*np.arange(n+1))
    term2 = np.log(a + 1/a)
    term3 = np.log(2*(1+f[0]*dt))
    return (term1 + term2 - term3) / dt

def zeroCouponTree(prds, maturity, tree):
    n = int(prds * maturity)
    rates = [np.array(x) for x in tree]
    x = 100 * np.ones(n + 1)
    lst = [x]
    i = n - 1
    while len(x) > 1:
        x = 0.5 * (x[:-1] + x[1:]) / (1 + rates[i] / prds)
        lst.insert(0, x)
        i -= 1
    return [list(x) for x in lst]



In [ ]:
# example parameters
sigma = 0.005       # Volatility
prds = 2            # Periods per year
total = 10          # Total periods
new_num = 6         # Number of bonds
zerom = 5           # Maturity of zero-coupon bond

# example bonds
bonds = [
        {"maturity": 1, "coupon": 0.02, "yld": 0.015},
        {"maturity": 2, "coupon": 0.02, "yld": 0.02},
        {"maturity": 3, "coupon": 0.02, "yld": 0.0225},
        {"maturity": 4, "coupon": 0.02, "yld": 0.024},
        {"maturity": 5, "coupon": 0.02, "yld": 0.025},
        {"maturity": 6, "coupon": 0.02, "yld": 0.026},
    ]

# number of grids
dt = 1/prds

spots = spot_rates(prds, bonds)
forwards = forward_rates(prds, spots)
phis = phi(sigma, prds, total, forwards)
tree = rateTree(spots[0], sigma, dt, phis)

# Annual Short Rate Tree
df_tree = pd.DataFrame(tree, index=['year ' + str(i/2) for i in range(total+1)]).transpose()
df_tree

,year 0.5,year 1.0,year 1.5,year 2.0,year 2.5,year 3.0,year 3.5,year 4.0,year 4.5,year 5.0,year 5.5
0,0.014804,0.020787,0.029190,0.036014,0.041274,0.045864,0.049792,0.053694,0.057570,0.061748,0.066228
1,NaN,0.013716,0.022119,0.028943,0.034202,0.038793,0.042721,0.046623,0.050499,0.054677,0.059157
2,NaN,NaN,0.015048,0.021872,0.027131,0.031722,0.035650,0.039552,0.043428,0.047606,0.052086
3,NaN,NaN,NaN,0.014801,0.020060,0.024651,0.028579,0.032481,0.036357,0.040535,0.045015
4,NaN,NaN,NaN,NaN,0.012989,0.017580,0.021508,0.025410,0.029285,0.033464,0.037944
5,NaN,NaN,NaN,NaN,NaN,0.010509,0.014437,0.018339,0.022214,0.026393,0.030873
6,NaN,NaN,NaN,NaN,NaN,NaN,0.007366,0.011267,0.015143,0.019322,0.023802
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004196,0.008072,0.012251,0.016731
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001001,0.005180,0.009660
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.001891,0.002589


In [ ]:
# Zero Coupon Bond Price
zero = zeroCouponTree(prds, zerom, tree)
df_zero = pd.DataFrame(zero, index=['year ' + str(i/2) for i in range(total+1)]).transpose()
df_zero

df_zero

,year 0.0,year 0.5,year 1.0,year 1.5,year 2.0,year 2.5,year 3.0,year 3.5,year 4.0,year 4.5,year 5.0
0,88.293754,87.550514,87.228550,87.425470,88.073957,89.113754,90.527509,92.301373,94.453153,97.005054,100.0
1,NaN,90.344054,89.692424,89.577807,89.925559,90.669284,91.787154,93.261184,95.105612,97.338892,100.0
2,NaN,NaN,92.234870,91.790917,91.822736,92.257511,93.068784,94.234349,95.764855,97.675036,100.0
3,NaN,NaN,NaN,94.066736,93.766773,93.879241,94.372862,95.221101,96.430977,98.013509,100.0
4,NaN,NaN,NaN,NaN,95.758998,95.535299,95.699861,96.221678,97.104072,98.354337,100.0
5,NaN,NaN,NaN,NaN,NaN,97.226538,97.050267,97.236323,97.784240,98.697543,100.0
6,NaN,NaN,NaN,NaN,NaN,NaN,98.424576,98.265284,98.471579,99.043152,100.0
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.308815,99.166191,99.391191,100.0
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.868179,99.741684,100.0
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.094658,100.0


In [ ]:
price = zero[0][0]
rate = ((100/price)**(1/(prds*zerom)) - 1)*prds

print('The Date-0 bond price is', f"${price:,.2f}", end='.\n')
print('The spot rate is', f"{rate:,.2f}", end='.\n')

The Date-0 bond price is $88.29.
The spot rate is 0.03.
